In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
import string

In [2]:
# import nltk
# import os

# nltk_data_path = "/home/tunar_ssp/nltk_data"
# os.makedirs(nltk_data_path, exist_ok=True)
# nltk.data.path.insert(0, nltk_data_path)

# nltk.download("stopwords", download_dir=nltk_data_path)
# nltk.download("punkt_tab", download_dir=nltk_data_path)

In [3]:
dataframe_emails=pd.read_csv('emails.csv')
dataframe_emails.head(10)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


In [4]:
num_email=len(dataframe_emails)

print(f'Number of emails : {num_email}')
print(f'Percantage of spam emails : {dataframe_emails.spam.sum()/num_email *100:.2f} % and ham : {(1-dataframe_emails.spam.sum()/num_email) *100:.2f}%')

Number of emails : 5728
Percantage of spam emails : 23.88 % and ham : 76.12%


In [5]:
dataframe_emails.text[0][:8]

'Subject:'

In [6]:
def preprocess_emails(df):
    df = df.sample(frac=1,ignore_index=True,random_state=42)
    # remove 'Subject:' and convert into numpy array
    X=df.text.apply(lambda x : x[9:]).to_numpy()
    Y=df.spam.to_numpy()

    return X,Y
    


In [7]:
X, Y = preprocess_emails(dataframe_emails)

In [8]:
print(X[123])
print(Y[123])

re : mit financial engineering pro - seminar  claudio ,  i have done it twice in the past . i shall be glad to help again .  vince  from : claudio ribeiro @ enron 11 / 21 / 2000 08 : 32 am  to : vince j kaminski / hou / ect @ ect  cc :  subject : re : mit financial engineering pro - seminar  vince :  it was stewart myers , head of the financial engineering track . he did not  contact me directly , he contacted josef ( an mit student that spent the summer  with enron and received an offer to come to work with us ) .  josef wanted to know who could make the commitment of sponsoring ( sending the  problem ) .  claudio  vince j kaminski @ ect  11 / 21 / 2000 08 : 16 am  to : claudio ribeiro / corp / enron @ enron  cc :  subject : re : mit financial engineering pro - seminar  claudio .  who was the professor who came up with the request ?  vince  from : claudio ribeiro @ enron 11 / 20 / 2000 05 : 04 pm  to : vince j kaminski / hou / ect @ ect  cc : joseph hrgovcic / hou / ect @ ect  subject

In [9]:
def preprocess_text(X):
    stop = set(stopwords.words('english') + list(string.punctuation))

    if isinstance(X, str):
        X = np.array([X])

    X_preprocessed = []

    for i, email in enumerate(X):
        email = np.array([i.lower() for i in word_tokenize(email) if i.lower() not in stop]).astype(X.dtype)
        X_preprocessed.append(email)
        
    if len(X) == 1:
        return X_preprocessed[0]
    return X_preprocessed

In [10]:
X_ready= preprocess_text(X)

In [ ]:
# The percentage you want
Train_percentage=80

Train_size= int(len(X_ready) *Train_percentage / 100  )

X_Train=X_ready[: Train_size]
Y_Train=Y[: Train_size]

X_Test=X_ready[Train_size:]
Y_Test=Y[Train_size:]



In [12]:
print(f"Proportion of spam in train dataset: {sum(Y_Train == 1)/len(Y_Train):.2f}")
print(f"Proportion of spam in test dataset: {sum(Y_Test == 1)/len(Y_Test):.2f}")

Proportion of spam in train dataset: 0.24
Proportion of spam in test dataset: 0.21


In [13]:
def get_word_frequency(X,Y):
    word_dict={}
    num_emails=len(Y)
    for i in range (num_emails):
        email=X[i]
        
        cls=Y[i]

        email=set(email)


        for word in email:


            if word not in word_dict.keys():
                word_dict[word]={"spam":1, "ham":1}
            if cls==1:
                word_dict[word]['spam']+=1
            else:
                word_dict[word]['ham']+=1
            
    return word_dict


In [14]:
word_frequency=get_word_frequency(X_Train,Y_Train)
class_frequency = {'ham': sum(Y_Train == 0), 'spam': sum(Y_Train == 1)}

In [15]:
print(class_frequency)

{'ham': np.int64(4133), 'spam': np.int64(1308)}


In [16]:
proportion_spam = class_frequency['spam']/(class_frequency['ham'] + class_frequency['spam'])

print(f"The proportion of spam: {proportion_spam:.2f}")

The proportion of spam: 0.24


In [17]:
def prob_word_given_class(word, cls, word_frequency, class_frequency):
    amount_word_and_class = word_frequency[word][cls]
    p_word_given_class = amount_word_and_class/class_frequency[cls]

    return p_word_given_class


In [18]:
print(f"P(lottery | spam) = {prob_word_given_class('lottery', cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(lottery | ham) = {prob_word_given_class('lottery', cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(schedule | spam) = {prob_word_given_class('schedule', cls = 'spam', word_frequency = word_frequency, class_frequency = class_frequency)}")
print(f"P(schedule | ham) = {prob_word_given_class('schedule', cls = 'ham', word_frequency = word_frequency, class_frequency = class_frequency)}")

P(lottery | spam) = 0.006880733944954129
P(lottery | ham) = 0.00024195499637067505
P(schedule | spam) = 0.008409785932721712
P(schedule | ham) = 0.10307282845390757


In [19]:
def prob_email_given_class(treated_email, cls, word_frequency, class_frequency):
    prob = 1


    for word in treated_email:
        if word in word_frequency.keys(): 
            
            prob *= word_frequency[word][cls] / class_frequency[cls]

    return prob

In [20]:
def naive_bayes(treated_email, word_frequency, class_frequency, return_likelihood = False):

    prob_email_given_spam = prob_email_given_class(treated_email, "spam" , word_frequency, class_frequency)

    prob_email_given_ham = prob_email_given_class(treated_email, "ham", word_frequency, class_frequency)

    p_spam = class_frequency['spam']/( class_frequency['spam']+ class_frequency['ham'])

    p_ham =  class_frequency['ham']/( class_frequency['spam']+ class_frequency['ham'])

    spam_likelihood = prob_email_given_spam * p_spam

    ham_likelihood =  prob_email_given_ham * p_ham


    if return_likelihood == True:
        return (spam_likelihood, ham_likelihood)
    elif spam_likelihood >= ham_likelihood:
        return 1
    else:
        return 0

In [21]:
Y_pred = []



for email in X_Test:
    prediction = naive_bayes(email, word_frequency, class_frequency)
    Y_pred.append(prediction)

In [22]:
def get_true_pos_or_neg(Y_Pred,Y_Test,l):
    if len(Y_Pred) != len(Y_Test):
        return "They Must be same size!!!!!"
    true_positives=0

    if l==1 or l == 0:
        pass
    else:
        return "You must give 0 or 1 (0 for True Negative 1 for True Positive)" 

    for i in range(len(Y_Pred)):
        if Y_pred[i]==Y_Test[i] and Y_pred[i]==l :
            true_positives+=1
    return true_positives


In [23]:
true_positive=get_true_pos_or_neg(Y_pred,Y_Test,1)
true_negative=get_true_pos_or_neg(Y_pred,Y_Test,0)
accuracy = (true_positive + true_negative)/len(Y_Test)
print(f"True_positive is:{true_positive} and True Negative is:{true_negative} and Total:{len(Y_Test)}")
print(f"Accuracy is: {accuracy * 100 :.4f}%")


True_positive is:60 and True Negative is:187 and Total:287
Accuracy is: 86.0627%
